In [1]:
import pandas as pd
from sklearn import model_selection
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import threading
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
%matplotlib inline

In [102]:
import imblearn

In [2]:
train_df = pd.read_csv('train_small.csv', header=None)
test_df = pd.read_csv('test_small.csv')

In [76]:
irregulars = [14, 103, 128, 136, 152, 155, 160, 166, 177, 179, 182, 198]
train_df = train_df.iloc[:,[0, 1]+irregulars]
test_df = test_df.iloc[:, np.array([1] + irregulars)-1]
train_df

,0,1,14,103,128,136,152,155,160,166,177,179,182,198
0,train_1,0,14.0239,8.4068,14.5697,1.9568,20.6294,16.7242,22.3949,-5.0121,10.4968,16.5721,-5.5937,8.1267
1,train_2,0,14.1929,12.6317,14.1978,3.9358,14.3330,14.1479,33.8820,-1.0410,10.9202,8.8438,-4.2935,-6.5213
2,train_4,0,13.8481,12.3562,12.5815,-6.8304,20.1461,19.8234,27.0846,6.5769,16.3309,13.5261,-7.6938,3.9267
3,train_5,0,13.6380,19.8236,13.0031,-8.8256,19.5171,15.2331,24.2054,1.3847,9.2654,10.7687,7.1968,-3.6241
4,train_6,0,14.1629,17.0682,12.1880,-13.2871,11.6169,13.1753,21.9814,6.1213,12.7942,8.1735,-8.1416,9.1104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70052,train_199983,0,14.4175,9.4990,13.6724,-11.8286,15.6931,21.1326,22.5358,-2.9164,16.7099,12.3458,-0.1731,3.5126
70053,train_199987,0,14.2372,2.0500,13.3528,-13.6591,12.7498,14.6672,10.6501,-7.8169,9.6030,12.5392,-8.7072,-3.5687
70054,train_199993,0,14.3334,9.9091,14.1692,-0.8046,17.2302,17.8464,19.5502,-8.2583,12.3769,10.5019,-2.8878,6.7302
70055,train_199996,0,14.4131,9.5501,12.2389,-6.0942,18.7807,17.8085,15.3500,-9.7142,11.8542,11.1395,2.5058,-2.1651


In [64]:
print(train_df.mean())
print(test_df.mean())

1       0.099733
14     14.023154
103    13.439792
128    13.200582
136    -4.815448
152    16.817737
155    16.717410
160    17.932712
166    -2.861032
177    11.636009
179    11.884891
182    -2.745573
198     2.298631
dtype: float64
var_12     14.024578
var_101    13.375967
var_126    13.204643
var_134    -4.844257
var_150    16.823070
var_153    16.730608
var_158    17.896491
var_164    -2.876100
var_175    11.629153
var_177    11.887321
var_180    -2.745725
var_196     2.292337
dtype: float64


In [18]:
n_train = 60000
train_targets = train_df.iloc[:n_train,1]
train_data = train_df.iloc[:n_train,2:]
test_targets = train_df.iloc[n_train:,1]
test_data = train_df.iloc[n_train:,2:]


def auc2(m, train, test): 
    return (metrics.roc_auc_score(train_targets,m.predict(train)),
                            metrics.roc_auc_score(test_targets,m.predict(test)))

In [103]:
# Organise the data into numpy arrays so that it is handled correctly by imblearn
nump_train_data = train_data.to_numpy()
nump_train_data = nump_train_data.astype(float)
nump_train_targets = train_targets.to_numpy()
nump_train_targets = nump_train_targets.astype(int)

nump_test_data = test_data.to_numpy()
nump_test_data = nump_test_data.astype(float)
nump_test_targets = test_targets.to_numpy()
nump_test_targets = nump_test_targets.astype(int)

rus = imblearn.under_sampling.RandomUnderSampler(sampling_strategy='auto',return_indices=True)
X_rus, y_rus, id_rus = rus.fit_sample(nump_train_data, nump_train_targets)

In [25]:
lg = lgb.LGBMRegressor(silent=False)
param_dist = {"max_depth": [1,2, 3],
              "learning_rate" : [0.1,0.5,1],
              # "num_leaves": [10,15,20],
              "num_leaves": [2,3],
              "n_estimators": [300]
             }
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
grid_search.fit(train_data,train_targets)
grid_search.best_estimator_

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  50 out of  54 | elapsed:   28.4s remaining:    2.2s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:   30.0s finished


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.5, max_depth=1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=2, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [30]:
lg

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.5, max_depth=10,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=300, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [26]:
auc2(grid_search.best_estimator_, train_data, test_data)

(0.941760012507842, 0.8508816764850232)

### Best predictors
500 rows 150 estimators (12s) : depth 3; rate 0.5; 10 leaves;

1000 rows 200 estimators (19s) : depth 2; rate 0.1; 10 leaves; 0.7361517358288425 auc (1 with training set)

10000 rows 300 estimators (1min1s) : depth 1; rate 0.5; 10 leaves; 0.8508816764850232 auc (0.941760012507842 with training set)

Does number of leaves impact ? NO default 31

60000 rows; 300 estiators; depth 1; rate 0.5; 10 leaves : (0.8986034603281355, 0.8795559277631042) (train then test)
60000 rows; 1000 estiators; depth 1; rate 0.5; 10 leaves : (0.9183773853908943, 0.8924150185745882) (train then test)
#### Shows that overfitting is not good ? Or just harder to have high auc with bigger amount of data.
60000 rows; 10000 estiators; depth 1; rate 0.5; 10 leaves : (0.9433394334525005, 0.8876156885481759) (train then test)
#### Now it seems clear

In [5]:
lggood = lgb.LGBMRegressor(silent=False, max_depth=1, learning_rate=0.5, n_estimators=1000)
start = time.time()
lggood.fit(train_data,train_targets)
end = time.time()
print(end-start)
auc2(lggood, train_data, test_data)

7.802568674087524


(0.9183773853908943, 0.8924150185745882)

In [101]:
print(lggood)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.5, max_depth=1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=1000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              scale_pos_weight=100, silent=False, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)


rate 0.5 1000 estim : (0.9183773853908943, 0.8924150185745882) in 6.60472559928894s
rate 0.3 1000 estim : (0.9158483146682737, 0.8915313835384053) in 5.39690375328064s
0.3 800 5.174034118652344
(0.91017955230423, 0.8890010522021087)
0.5 1500 8.940380811691284
(0.9225535053779779, 0.892192660353454)

#### Overfitting : 
rate 0.5 1000 estim depth 2 : (0.9736699095251268, 0.8562574888874572) 9.129353523254395s
rate 0.5 300 estim depth 2 : (0.9332266802884277, 0.8697272863922353) kinda 3.4s
rate 1 200 est depth 5 : (0.9997954560831425, 0.7009104769267109) in 5.8162806034088135
rate 1 200 est depth 5 : (1.0, 0.6860658377890871) in 26.31138586997986s


#### Mmmmmh
rate 0.1 200 est depth 5 : (0.9999726974419348, 0.8615919388434367) in 23.673893928527832s
0.03 1000 est : 25.953459978103638s (0.9924414078232998, 0.8748606369043784)
0.01 : 25.04612970352173 (0.9542057987408105, 0.8484477871545448)
0.01 3000 est : (0.9925341998563096, 0.8746814404432133) 78.72248482704163s (meh)

#### To few estimators in this case (?):
rate 0.1 300 estim depth 2 : (0.8918921360973638, 0.8568826902016364) 3.3066024780273438s
rate 0.1 1000 estim depth 2 : (0.9356584753554953, 0.8837796602890337) 8.71259069442749s

In [113]:
lgrus = lgb.LGBMRegressor(silent=False, max_depth=1, learning_rate=0.1, n_estimators=3000)
start = time.time()
lgrus.fit(X_rus,y_rus)
end = time.time()
print(end-start)
auc2(lgrus, train_data, test_data)

4.418010711669922


(0.9144309306569556, 0.888129979170693)

#### Balancing data
scale_pos_weight useless
with RUS:
rate 0.5 1000 estim : 1.704514980316162s : (0.9156733819245538, 0.8793312074556034)
rate 0.1 3000 estim : (0.9144309306569556, 0.888129979170693) 4.4809534549713135s
rate 0.1 4000 estim : (0.9172283755513937, 0.887266314501063)
rate 0.08 5000 estim : 7.250913619995117s (0.9161398215981195, 0.8879848182267172)

In [15]:
class Bag_pred:
    def __init__(self, predictors=[], weights=None, scatter=True):
        self.predictors = predictors
        if weights==None:
            self.weights = list(np.ones(len(predictors)))
        else:
            self.weights = list(weights)
        self.scatter = scatter
            
    def predict(self, Y):
        predictions = [pred.predict(Y) for pred in self.predictors]
        weights = np.array(self.weights)/sum(self.weights)
        total = weights[0]*np.array(self.predictors[0].predict(Y))
        total.shape = len(total)
        for pred, weight in zip(predictions, weights[1:]):
            prediction = weight*np.array(pred)
            prediction.shape = len(prediction)
            total += prediction
        return total
    
    
    def predict1(self, Y):
        
        def Lambda(predictor,Y,predicts,i):
            predicts[i] = predictor.predict(Y)
        
        threads = []
        predictions = 5*[0]
        for i, pred in enumerate(self.predictors):
            threads.append(threading.Thread(target=Lambda, args=(pred,Y,predictions,i)))
            threads[i].start()
        for i, _ in enumerate(self.predictors):
            threads[i].join()
        
        weights = np.array(self.weights)/sum(self.weights)
        total = weights[0]*np.array(predictions[0])
        total.shape = len(total)
        for pred, weight in zip(predictions, weights[1:]):
            prediction = weight*np.array(pred)
            prediction.shape = len(prediction)
            total += prediction
        return total
    
    def fit(self, X, y):
        if(self.scatter):
            factor = int(X/len(self.predictors))
            for i, pred in in self.predictors:
                low, high = factor*i, factor*(i+1)
                yi = train_df.iloc[low:high,1]
                Xi = train_df.iloc[low:high,2:]
                pred.fit(Xi, yi)
        else:
            for pred in self.predictors:
                pred.fit(X,y)

            
    def append(self, pred, weight=1):
        self.predictors.append(pred)
        self.weights.append(weight)
            

In [21]:
# Let's BAG
Bag = Bag_pred()
start = time.time()
for i in range(6):
    low, high = 10000*i, 10000*(i+1)
    train_targetsi = train_df.iloc[low:high,1]
    train_datai = train_df.iloc[low:high,2:]
    
    def auci(m, train, test): 
        return (metrics.roc_auc_score(train_targetsi,m.predict(train)),
                metrics.roc_auc_score(test_targets,m.predict(test)))
    
    lgover = lgb.LGBMRegressor(silent=False, max_depth=5, learning_rate=0.035, n_estimators=200)
    #lgover.fit(train_datai, train_targetsi)
    #print(i, auci(lgover, train_datai, test_data))
    Bag.append(lgover)
Bag.fit(train_data, train_targets)
end = time.time()
print(end-start)
start = time.time()
print(auc2(Bag, train_data, test_data))
end = time.time()
print(end-start)

152.05464506149292
(0.9101142647942649, 0.8183159397882712)
23.003049850463867


In [ ]:
# No reinit of the bag ....
6 depth 5 r 0.035 n 200 (0.9345731839134271, 0.8709206768451114) = with n=400
6 depth 3 r 0.035 n 200 (0.9265064602771006, 0.8644267645858833)
6 depth 3 r 0.035 n 400 (0.9235606891946174, 0.8654291481457622) in 13s
6 depth 5 r 0.035 n 100 ~0.86
6 depth 5 r 0.035 n 50 (0.9311799920441334, 0.8681047477935966) in 6 + 43
6 depth 5 r 0.035 n 50

In [120]:
cbr = cb.CatBoostRegressor(silent=True, depth=1, learning_rate=0.3, iterations=1000)
# l2 leaf reg ??
start = time.time()
cbr.fit(train_data,train_targets)
end = time.time()
print(end-start)
auc2(cbr, train_data, test_data)

17.354048490524292


(0.9127614850298509, 0.8919345487341365)

rate 0.5 : (0.917555524639378, 0.8912588846657648) in 16.263858318328857s
rate 0.3 : (0.9127614850298509, 0.8919345487341365) in 14.216562747955322s

In [121]:
xgbr = xgb.XGBRegressor(silent=False, max_depth=1, learning_rate=0.5, n_estimators=300, objective='binary:logistic')
# min_child_weight ??
start = time.time()
xgbr.fit(train_data,train_targets)
end = time.time()
print(end-start)
auc2(xgbr, train_data, test_data)

C:\Users\LAMERAND\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\LAMERAND\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


90.60874819755554


(0.9068527447700975, 0.8810668040971461)

In [17]:
xgbr

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.5, max_delta_step=0,
             max_depth=1, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='binary:logistic',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=False, subsample=1, verbosity=1)

300 estimators : (0.9068527447700975, 0.8810668040971461) in 76.34813570976257
500 estimators : (0.9178513085660571, 0.8891569499022955) in 137.13004994392395

XGBoost is not very Parallel with stamps

In [10]:
lgr = lgb.LGBMRegressor(silent=False)
param_dist = {"max_depth": [1],
              "learning_rate" : np.arange(0.2,0.8,0.1),
              # "num_leaves": [10,15,20],
              #"num_leaves": [2,3],
              "n_estimators": [1000]
             }
grid_search = GridSearchCV(lgr, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
start = time.time()
grid_search.fit(train_data,train_targets)
end = time.time()
print(end-start)
print(grid_search.best_estimator_.learning_rate)
print(auc2(grid_search.best_estimator_, train_data, test_data))

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   38.2s
[Parallel(n_jobs=-1)]: Done  11 out of  21 | elapsed:  1.2min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  16 out of  21 | elapsed:  1.2min remaining:   23.2s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.6min finished


106.87924218177795


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.30000000000000004,
              max_depth=1, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=1000, n_jobs=-1, num_leaves=31,
              objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
              silent=False, subsample=1.0, subsample_for_bin=200000,
              subsample_freq=0)

Learning rate does not seem to impact much

In [16]:
lgr = lgb.LGBMRegressor(silent=False)
param_dist = {"max_depth": [1],
              "learning_rate" : [0.5],
              # "num_leaves": [10,15,20],
              #"num_leaves": [2,3],
              "n_estimators": [800, 1000, 1500, 2000, 3000]
             }
grid_search = GridSearchCV(lgr, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
start = time.time()
grid_search.fit(train_data,train_targets)
end = time.time()
print(end-start)
print(grid_search.best_estimator_.n_estimators)
print(auc2(grid_search.best_estimator_, train_data, test_data))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  15 | elapsed:   52.2s remaining:  2.4min
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  1.1min remaining:   58.6s
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  1.8min remaining:   27.2s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.2min finished


139.06517004966736
800
(0.9158483146682737, 0.8915313835384053)


neither a precise number estims ?

In [132]:
from mlxtend.classifier import StackingCVClassifier


meta = DecisionTreeRegressor()
sclf = StackingCVClassifier(classifiers=[lggood, xgbr, cbr, lgrus],
                            meta_classifier=meta,
                            random_state=785)
sclf.fit(train_data,train_targets)

StackingCVClassifier(classifiers=[LGBMRegressor(boosting_type='gbdt',
                                                class_weight=None,
                                                colsample_bytree=1.0,
                                                importance_type='split',
                                                learning_rate=0.5, max_depth=1,
                                                min_child_samples=20,
                                                min_child_weight=0.001,
                                                min_split_gain=0.0,
                                                n_estimators=1000, n_jobs=-1,
                                                num_leaves=31, objective=None,
                                                random_state=None,
                                                reg_alpha=0.0, reg_lambda=0.0,
                                                silent=False, subsampl...
                                                     

In [136]:
sclf.meta_classifier.fit(train_data,train_targets)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [138]:
sclf.meta_classifier.tree_.

In [140]:
auc2(sclf.meta_classifier, train_data, test_data)

(1.0, 0.5497382915673518)

In [145]:
stack = [lggood, xgbr, cbr, lgrus, Bag]
mat = []
for model in stack:
    mat.append(model.predict(train_data))
meta_df = pd.DataFrame(np.transpose(np.array(mat)))

In [164]:
from sklearn.linear_model import LogisticRegression
meta = LogisticRegression()
meta.fit(meta_df, train_targets)

C:\Users\LAMERAND\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [150]:
stack = [lggood, xgbr, cbr, lgrus, Bag]
mat = []
for model in stack:
    mat.append(model.predict(test_data))
meta_df_test = pd.DataFrame(np.transpose(np.array(mat)))

In [166]:
print(meta_df)

              0         1         2         3         4
0      0.360781  0.258471  0.314487  0.862897  0.151870
1     -0.052274  0.014589 -0.052279  0.095015  0.054187
2      0.151852  0.085439  0.153031  0.477342  0.081380
3      0.056884  0.031288  0.079956  0.330975  0.080134
4      0.147655  0.227023  0.153765  0.544773  0.102666
...         ...       ...       ...       ...       ...
59995  0.524123  0.547292  0.472190  0.889156  0.346260
59996 -0.082391  0.010415 -0.068166 -0.015968  0.042841
59997 -0.035736  0.015842 -0.018454  0.090210  0.046670
59998  0.694818  0.946699  0.624575  1.085728  0.376242
59999  0.143685  0.079833  0.153201  0.612880  0.153800

[60000 rows x 5 columns]


In [165]:
auc2(meta, meta_df, meta_df_test)

(0.7453738173688175, 0.6693325495501299)

In [158]:
estimator = meta
n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold


# The tree structure can be traversed to compute various properties such
# as the depth of each node and whether or not it is a leaf.
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]  # seed is the root node id and its parent depth
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    # If we have a test node
    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("The binary tree structure has %s nodes and has "
      "the following tree structure:"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snode=%s leaf node." % (node_depth[i] * "\t", i))
    else:
        print("%snode=%s test node: go to node %s if X[:, %s] <= %s else to "
              "node %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 feature[i],
                 threshold[i],
                 children_right[i],
                 ))
print()

# First let's retrieve the decision path of each sample. The decision_path
# method allows to retrieve the node indicator functions. A non zero element of
# indicator matrix at the position (i, j) indicates that the sample i goes
# through the node j.

node_indicator = estimator.decision_path(X_test)

# Similarly, we can also have the leaves ids reached by each sample.

leave_id = estimator.apply(X_test)

# Now, it's possible to get the tests that were used to predict a sample or
# a group of samples. First, let's make it for the sample.

sample_id = 0
node_index = node_indicator.indices[node_indicator.indptr[sample_id]:
                                    node_indicator.indptr[sample_id + 1]]

print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:
    if leave_id[sample_id] == node_id:
        continue

    if (X_test[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("decision id node %s : (X_test[%s, %s] (= %s) %s %s)"
          % (node_id,
             sample_id,
             feature[node_id],
             X_test[sample_id, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

# For a group of samples, we have the following common node.
sample_ids = [0, 1]
common_nodes = (node_indicator.toarray()[sample_ids].sum(axis=0) ==
                len(sample_ids))

common_node_id = np.arange(n_nodes)[common_nodes]

print("\nThe following samples %s share the node %s in the tree"
      % (sample_ids, common_node_id))
print("It is %s %% of all nodes." % (100 * len(common_node_id) / n_nodes,))

The binary tree structure has 3 nodes and has the following tree structure:
node=0 test node: go to node 1 if X[:, 4] <= 0.19285798817873 else to node 2.
	node=1 leaf node.
	node=2 leaf node.



NameError: name 'X_test' is not defined

In [12]:
start = time.time()
scores = model_selection.cross_val_score(lggood, train_data, train_targets, cv=7, scoring='roc_auc', verbose=Tue)
end = time.time()
print(end-start)
print(scores)

47.36787152290344
[0.88433053 0.88367097 0.89130639 0.88945922 0.88313811 0.89352368
 0.89327486]


In [13]:
print(scores.mean())

0.8883862521165146
